# Timer

In [1]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 10, 27, 17, 15

while True:
    dt = datetime.now()
    if dt.month >= month:
        if dt.day > day:
            break
        if dt.day == day and (dt.hour > hour or (dt.hour == hour and dt.minute > minute)):
            break
    sleep(60)

# Parameter optimization

In [2]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['PriceChange']
work_timeframe = '15m'
higher_timeframe = '4h'
opt_limit = 50
load = False

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {'PriceChange': {"low_price_quantile": [1 * (i + 1) for i in range(20)]}}

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Number of combinations is 20


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [03:07<00:00,  9.38s/it]


# Check global statistics

In [3]:
from glob import glob

ttype = 'buy'
pattern = ['PriceChange']
work_timeframe = '15m'
higher_timeframe = '4h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
stat.groupby(['PriceChange_low_price_quantile']).agg({'pct_right_forecast_avg': 'mean',
                                                             'pct_price_diff_avg': 'mean',
                                                             'forecast_rank': 'mean', 
                                                             'price_rank': 'mean', 
                                                             'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                                  ascending=False).head(20)

,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank,forecasts_num
PriceChange_low_price_quantile,,,,,
1,43.750000,1.978281,-38.958333,0.518229,11
4,54.731458,2.078594,-53.229687,2.909427,34
5,53.862917,1.194479,-57.597604,3.319167,42
2,46.136510,2.141823,-59.585260,0.714219,18
6,66.752552,2.393021,-60.205260,5.518646,52
3,48.125000,2.223177,-71.458333,0.624323,24
7,62.387135,1.245521,-97.709271,3.056927,62
9,62.993646,1.130885,-104.269010,5.529792,84
8,61.986771,1.929635,-105.313854,6.297604,69


# Save new config data to config file

In [6]:
from config_updater import ConfigUpdater

timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'PriceChange': {"low_price_quantile": [6]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)